In [ ]:
import numpy as np
from keras import utils
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
from keras.datasets import imdb 
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from keras.layers import LSTM


'''Рекуррентные нейронные сети.'''

# Реализация сети RNN на основе Numpy
timesteps = 100 # Число временных интервалов во вх посл-ти
input_features = 32 # Размерность пространства вх признаков
output_features = 64 # Размерность пространства вых признаков

# Входные данные: случайный шум для простоты примера
inputs = np.random.random((timesteps, input_features))

# Начальное состояние: вектор с нулевыми значениями элементов
state_t = np.zeros((output_features, ))

# Создание матриц со случайными весами
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features))

successive_outputs = []
for input_t in inputs:
    # Объед-ние вх данных с текущем состоянием (вых данными на пред. шаге)
    output_t = np.tanh(np.dot(W, input_t)+np.dot(U, state_t)+b)
    
    # Сохранение вых данных в список
    successive_outputs.append(output_t)

    # Обновление текущего состояния
    state_t = output_t

# Окончательный рез-т - 2мерный тензор (времен. интервалы, вх признаки)
final_output_sequence = np.concatenate(successive_outputs, axis=0)

'''Рекуррентный слой в Керас.'''

model =Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))
model.summary()

# Подготовка данных IMDB

max_features = 10000 # Кол-во слов рассм-х как признаки
maxlen = 500 # Обрезка текста после этого кол-ва слов
batch_size = 32 

print("Loading data")
(input_train, y_train), (input_test, y_test) = imdb.load_data(
    num_words=max_features)
print(len(input_train), "train sequences")
print(len(input_test), "test sequences")

print("pad sequences (samples x time)")
input_train = utils.data_utils.pad_sequences(input_train, maxlen=maxlen)
input_test = utils.data_utils.pad_sequences(input_test, maxlen=maxlen)
print("input_train shape:", input_train.shape)
print("input_test shape:", input_test.shape)

# Обучение модели со слоями Embedding и SimpleRNN

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc'])

history = model.fit(
    input_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.2
)

# Вывод результатов
import matplotlib.pyplot as plt 
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy ')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Реализация архитектуры LSTM в псевдокоде (1/2)
# псевдокод LSTM
# output_t = activation(dot(state_t, Uo)+dot(input_t, Wo)+dot(C_t, Vo)+ bo)
# i_t = activation(dot(state_t, Ui)+dot(input_t, Wl)+bi)
# f_t = activation(dot(state_t, Uf)+dot(input_t, Wf)+bf)
# k_t = activation(dot(state_t, Uk)+dot(input_t, Wk)+bk)
# c_t+1 = i_t+k_t+c_t*f_t

# использование слоя LSTM из Керас
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc'])

history = model.fit(
    input_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.2
)

# Plotting results
import matplotlib.pyplot as plt 
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy ')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()